# Empresas Tech de Madrid


In [1]:
from pymongo import MongoClient
import pandas as pd
import requests
import sys
sys.path.append('../src')
import func as fc

Generamos la conexsion con la base de datos 

In [2]:
client = MongoClient("localhost:27017")
db = client.get_database("ironhack")
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'ironhack')

In [3]:
db.list_collection_names()

['companies']

In [4]:
c = db.get_collection("companies")

In [5]:
c

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'ironhack'), 'companies')

In [6]:
listaempresas = list(db["companies"].find({},{"_id":0,"name":1,"offices":1}))

In [7]:
df_emp = pd.DataFrame(listaempresas)

In [8]:
df_emp;

In [9]:
df_emp = df_emp.explode('offices')

In [10]:
df_emp;

In [11]:
df_ofis = df_emp[["offices"]].apply(lambda x: x.offices, result_type="expand", axis=1)

In [12]:
df_ofis

,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude
0,,710 - 2nd Avenue,Suite 1100,98104,Seattle,WA,USA,47.603122,-122.333253
0,,270 Lafayette Street,Suite 505,10012,New York,NY,USA,40.723731,-73.996431
1,Headquarters,4900 Hopyard Rd.,Suite 310,94588,Pleasanton,CA,USA,37.692934,-121.904945
2,Headquarters,4900 Hopyard Rd,Suite 310,94588,Pleasanton,CA,USA,37.692934,-121.904945
3,None,135 Mississippi St,,94107,San Francisco,CA,USA,37.764726,-122.394523
...,...,...,...,...,...,...,...,...,...
18796,Indian Office,Near Tube Well(south),Patel Nagar,334003,Bikaner,None,IND,NaN,NaN
18797,Livingston,P.O. Box 2096,,07039,Livingston,NJ,USA,40.793024,-74.323554
18798,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18799,,,,,Birmingham,AL,USA,33.518885,-86.816068


In [13]:
df_mad = df_ofis[df_ofis['city'] == 'Madrid']

In [14]:
df_mad;

In [15]:
df_mad.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 86 entries, 48 to 18255
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   description   82 non-null     object 
 1   address1      86 non-null     object 
 2   address2      86 non-null     object 
 3   zip_code      86 non-null     object 
 4   city          86 non-null     object 
 5   state_code    1 non-null      object 
 6   country_code  86 non-null     object 
 7   latitude      53 non-null     float64
 8   longitude     53 non-null     float64
dtypes: float64(2), object(7)
memory usage: 6.7+ KB


In [16]:
df_madrid = df_emp.merge(df_mad, how='inner', left_index=True, right_index=True)

In [17]:
df_madrid;

In [18]:
df_madrid = df_emp.merge(df_mad, how='inner', left_index=True, right_index = True)
df_madrid;

In [19]:
df_madrid.drop_duplicates(subset='name', 
                          keep='first', 
                          inplace=True, 
                          ignore_index=False)
df_madrid;

## Aqui pordria limpiara la direccion y aplicar geocode func para sacara mas coordenadas, si tengo tiempo lo hare


In [20]:
#df_madrid['request'] = df_madrid.address1.apply(fc.geocode)

In [21]:
df_cerca = df_madrid
df_cerca;

In [22]:
df_cerca.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 80 entries, 48 to 18255
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   name          80 non-null     object 
 1   offices       80 non-null     object 
 2   description   76 non-null     object 
 3   address1      80 non-null     object 
 4   address2      80 non-null     object 
 5   zip_code      80 non-null     object 
 6   city          80 non-null     object 
 7   state_code    1 non-null      object 
 8   country_code  80 non-null     object 
 9   latitude      52 non-null     float64
 10  longitude     52 non-null     float64
dtypes: float64(2), object(9)
memory usage: 7.5+ KB


In [23]:
df_cerca = df_cerca[df_cerca['latitude'].notna()]


In [24]:
df_cerca.columns

Index(['name', 'offices', 'description', 'address1', 'address2', 'zip_code',
       'city', 'state_code', 'country_code', 'latitude', 'longitude'],
      dtype='object')

In [25]:
df_cerca = df_cerca.drop(['offices', 'description', 'address1', 'address2', 'zip_code', 'state_code', 'country_code', 'city'], axis=1)

In [26]:
df_cerca

,name,latitude,longitude
673,Fon,40.530724,-3.632687
2036,Buongiorno,40.448797,-3.669764
2195,Swotti,40.416741,-3.703250
2257,AsesorSeguros,40.497716,-3.647842
2349,eRepublik,40.430940,-3.695290
3038,NotePub,40.416741,-3.703250
3844,FeedzZ,40.472227,-3.653252
4560,Next Limit,40.455219,-3.650703
4614,Veedia,40.456586,-3.647013
4655,Cyloop,40.528845,-3.642991


In [27]:
df_cerca.assign(tipo='EmpresaTech')

,name,latitude,longitude,tipo
673,Fon,40.530724,-3.632687,EmpresaTech
2036,Buongiorno,40.448797,-3.669764,EmpresaTech
2195,Swotti,40.416741,-3.703250,EmpresaTech
2257,AsesorSeguros,40.497716,-3.647842,EmpresaTech
2349,eRepublik,40.430940,-3.695290,EmpresaTech
3038,NotePub,40.416741,-3.703250,EmpresaTech
3844,FeedzZ,40.472227,-3.653252,EmpresaTech
4560,Next Limit,40.455219,-3.650703,EmpresaTech
4614,Veedia,40.456586,-3.647013,EmpresaTech
4655,Cyloop,40.528845,-3.642991,EmpresaTech


In [28]:
df_cerca.to_csv('../data/empresasTech.csv')